In [1]:
import os

In [2]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\OneDrive\\Documents\\End-to-End-MLP-with-MLFlow'

### model training
**Configuramos las reglas del entrenamiento, parametros y columna objetivo**

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig: #<--- rutas para entrenar con los test.csv y train.csv segun lo definido en el config.yaml 
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float # <--- parametro alpha predefinido en params.yaml
    l1_ratio: float # <--- parametro l1 predefinido en mi params.yaml
    target_column: str # <---columna objetivo segun l odefinamos en schema.yaml ( Quality of wine)

**Creamos el ConfigurationManager**

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(  #<--- vemos que leerá schema, parametros y config.
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

**Definamos los Components de mi model trainer**

In [8]:
import pandas as pd #<--- para leer la data obviamente
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet #<--- cargo sklean para usar elasticNet
import joblib #<--- salvamos el modelo

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

            # Separamos (x,y) para train y test usando drops de columna objetivo.
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

            #inicializamos El modelo ElasticNet 
        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name)) #<--- salvamos el modelo.



**configuramos el pipeline y entrenamos el modelo para luego guardarlo**
(Este luego irá a /pipeline stage_04_model_trainer.py)

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-29 00:25:23,206: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-29 00:25:23,210: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-29 00:25:23,211: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-29 00:25:23,214: INFO: common: created directory at: artifacts]
[2023-09-29 00:25:23,216: INFO: common: created directory at: artifacts/model_trainer]
